# Computing discrete grad, curl and div in STRUPHY 

In [1]:
import numpy as np
import sympy as sp

import hylife.utilitis_FEEC.derivatives as der
import hylife.utilitis_FEEC.projectors_global as proj_glob

import hylife.utilitis_FEEC.bsplines as bsp

## Derivative and commuting property in 1D

In [2]:
# ----------------------
# function to be derived
# ----------------------
fun  = lambda xi1 : np.sin( xi1 )
Dfun = lambda xi1 : np.cos( xi1 )

#-----------------
# Create the grid:
#-----------------
# side lengths of logical cube
L = 2*np.pi 

# spline degrees
p = 3   

# periodic boundary conditions (use 'False' if clamped)
bc = True 

# loop over different number of elements (convergence test)
Nel_cases = [16]

# loop over different number of quadrature points per element
Nq_cases = [1, 2, 3, 4, 5, 6, 7, 8]

for Nel in Nel_cases:
    
    print('Nel=', Nel)
    
    # element boundaries
    el_b = np.linspace(0., L, Nel + 1) 

    # knot sequences
    T = bsp.make_knots(el_b, p, bc)
    
    for Nq in Nq_cases:

        # create an instance of the projector class
        obj = proj_glob.projectors_1d(T, p, bc, Nq)

        # compute coefficients
        coeffs_0 = obj.PI_0(fun)
        coeffs_1 = obj.PI_1(Dfun)

        # Build gradient matrix
        grad = der.GRAD_1d(T, p, bc)

        # Compute discrete derivative
        Dfun_h = np.dot(grad, coeffs_0)

        # Test commuting property
        print( 'Nq=', Nq, np.max( np.abs(coeffs_1-Dfun_h) ) )
        
    print('')

Nel= 16
Nq= 1 0.002534351944482871
Nq= 2 2.1721354222514755e-06
Nq= 3 7.180471217616002e-10
Nq= 4 1.2567724638756772e-13
Nq= 5 4.163336342344337e-16
Nq= 6 4.440892098500626e-16
Nq= 7 4.440892098500626e-16
Nq= 8 4.163336342344337e-16



## Grad, curl and div and commuting properties in 3D

In [11]:
# -----------------------
# functions to be derived
# -----------------------
fun1    = lambda xi1, xi2, xi3 : np.sin(xi1)*np.sin(xi2)*np.sin(xi3)
D1fun1  = lambda xi1, xi2, xi3 : np.cos(xi1)*np.sin(xi2)*np.sin(xi3)
D2fun1  = lambda xi1, xi2, xi3 : np.sin(xi1)*np.cos(xi2)*np.sin(xi3)
D3fun1  = lambda xi1, xi2, xi3 : np.sin(xi1)*np.sin(xi2)*np.cos(xi3)

fun2    = lambda xi1, xi2, xi3 :   np.sin(2*xi1)*np.sin(2*xi2)*np.sin(2*xi3)
D1fun2  = lambda xi1, xi2, xi3 : 2*np.cos(2*xi1)*np.sin(2*xi2)*np.sin(2*xi3)
D2fun2  = lambda xi1, xi2, xi3 : 2*np.sin(2*xi1)*np.cos(2*xi2)*np.sin(2*xi3)
D3fun2  = lambda xi1, xi2, xi3 : 2*np.sin(2*xi1)*np.sin(2*xi2)*np.cos(2*xi3)

fun3    = lambda xi1, xi2, xi3 :   np.sin(3*xi1)*np.sin(3*xi2)*np.sin(3*xi3)
D1fun3  = lambda xi1, xi2, xi3 : 3*np.cos(3*xi1)*np.sin(3*xi2)*np.sin(3*xi3)
D2fun3  = lambda xi1, xi2, xi3 : 3*np.sin(3*xi1)*np.cos(3*xi2)*np.sin(3*xi3)
D3fun3  = lambda xi1, xi2, xi3 : 3*np.sin(3*xi1)*np.sin(3*xi2)*np.cos(3*xi3)

#-----------------
# Create the grid:
#-----------------
# side lengths of logical cube
L = [2*np.pi, 2*np.pi , 2*np.pi] 

# spline degrees
p = [1, 2, 3]   

# periodic boundary conditions (use 'False' if clamped)
bc = [True, False, True] 

# FOR TESTING: loop over different number of elements (convergence test)
Nel_cases = [16]

# FOR TESTING: loop over different number of quadrature points per element
Nq_cases = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for Nel in Nel_cases:
    
    print('Nel=', Nel)
    
    # number of elements
    Nel = [8, 16, 32]   

    # element boundaries
    el_b = [np.linspace(0., L_i, Nel_i + 1) for L_i, Nel_i in zip(L, Nel)] 

    # knot sequences
    T = [bsp.make_knots(el_b_i, p_i, bc_i) for el_b_i, p_i, bc_i in zip(el_b, p, bc)]
    
    for Nq in Nq_cases:
        
        # number of elements
        Nq = [Nq, Nq, Nq]

        # create an instance of the projector class
        obj = proj_glob.projectors_3d(T, p, bc, Nq)

        # ---------------------------------------------
        # create LU decomposition of necessary matrices
        # ---------------------------------------------
        #obj.assemble_V0()   # for PI_0
        obj.assemble_V1()   # for PI_1
        obj.assemble_V2()   # for PI_2
        #obj.assemble_V3()   # for PI_3

        # --------------------
        # compute projections:
        # --------------------
        # for grad commutation:
        #coeffs_0  = obj.PI_0( fun1 )
        #coeffs_11 = obj.PI_11( D1fun1 )
        #coeffs_12 = obj.PI_12( D2fun1 )
        #coeffs_13 = obj.PI_13( D3fun1 )
        
        # for curl commutation:
        coeffs_11 = obj.PI_11( fun1 )
        coeffs_12 = obj.PI_12( fun2 )
        coeffs_13 = obj.PI_13( fun3 )
        coeffs_21 = obj.PI_21( lambda xi1, xi2, xi3 : D2fun3(xi1, xi2, xi3) 
                                                    - D3fun2(xi1, xi2, xi3) )
        coeffs_22 = obj.PI_22( lambda xi1, xi2, xi3 : D3fun1(xi1, xi2, xi3)
                                                    - D1fun3(xi1, xi2, xi3) )
        coeffs_23 = obj.PI_23( lambda xi1, xi2, xi3 : D1fun2(xi1, xi2, xi3)
                                                    - D2fun1(xi1, xi2, xi3) )
        
        # for div commutation:
        #coeffs_21 = obj.PI_21( fun1 )
        #coeffs_22 = obj.PI_22( fun2 )
        #coeffs_23 = obj.PI_23( fun3 )
        #coeffs_3  = obj.PI_3( lambda xi1, xi2, xi3 : D1fun1(xi1, xi2, xi3)
        #                                           + D2fun2(xi1, xi2, xi3)
        #                                           + D3fun3(xi1, xi2, xi3) )
        
        # create an instance of the derivatives class
        obj_der = der.discrete_derivatives(T, p, bc)

        # Build gradient/curl/div matrices
        #grad = obj_der.GRAD_3d()
        curl = obj_der.CURL_3d()
        #div  = obj_der.DIV_3d()

        # Compute discrete derivative
        #Dfun_h = grad.dot( coeffs_0.flatten() )
        Dfun_h = curl.dot( np.concatenate( ( coeffs_11.flatten(), 
                                             coeffs_12.flatten(), 
                                             coeffs_13.flatten() ) ) )
        #Dfun_h = div.dot( np.concatenate( ( coeffs_21.flatten(), 
        #                                    coeffs_22.flatten(), 
        #                                    coeffs_23.flatten() ) ) )
        
        # Projection of true gradient/curl/div
        #Dfun_proj = coeffs_0.flatten()
        #Dfun_proj = np.concatenate( ( coeffs_11.flatten(), 
        #                              coeffs_12.flatten(), 
        #                              coeffs_13.flatten() ) )
        Dfun_proj = np.concatenate( ( coeffs_21.flatten(), 
                                      coeffs_22.flatten(), 
                                      coeffs_23.flatten() ) )
        #Dfun_proj = coeffs_3.flatten()

        # Test commuting property
        print( 'Nq=', Nq, np.max( np.abs( Dfun_proj-Dfun_h ) ) )
        
    print('')

Nel= 16
Nq= [1, 1, 1] 0.11290397353203263
Nq= [2, 2, 2] 0.0034775287862933157
Nq= [3, 3, 3] 4.1904359387945345e-05
Nq= [4, 4, 4] 2.659366619695014e-07
Nq= [5, 5, 5] 1.0414434314931498e-09
Nq= [6, 6, 6] 2.7684521342052903e-12
Nq= [7, 7, 7] 5.717648576819556e-15
Nq= [8, 8, 8] 1.582067810090848e-15
Nq= [9, 9, 9] 2.4980018054066022e-15
Nq= [10, 10, 10] 2.1094237467877974e-15

